# 表示文本

如果我们想用神经网络解决自然语言处理 (NLP) 任务，我们需要某种方式将文本表示为张量。计算机已经将文本字符表示为使用 ASCII 或 UTF-8 等编码映射到屏幕上字体的数字。

![显示将字符映射到 ASCII 和二进制表示的图表的图像](./images/ascii-character-map.png)

我们了解每个字母 ** 代表** 的含义，以及所有字符如何组合在一起形成句子的单词。但是，计算机本身并没有这样的理解，神经网络必须在训练过程中学习其含义。

因此，我们可以在表示文本时使用不同的方法：
* **字符级表示**，当我们通过将每个字符视为数字来表示文本时。鉴于我们的文本语料库中有 $C$ 不同的字符，单词 *Hello* 将由 $5\times C$ 张量表示。每个字母将对应于 one-hot 编码中的一个张量列。
* **词级表示**，其中我们为文本中的所有单词创建一个**词汇表**，然后使用one-hot编码表示单词。这种方法在某种程度上更好，因为每个字母本身并没有多大意义，因此通过使用更高级别的语义概念——单词——我们简化了神经网络的任务。然而，鉴于字典很大，我们需要处理高维稀疏张量。

让我们首先安装一些我们将在本模块中使用的必需 Python 包。

In [1]:
!pip3 install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached gensim-3.8.3-cp38-cp38-macosx_10_9_x86_64.whl (24.2 MB)
  Using cached huggingface-0.0.1-py3-none-any.whl (2.5 kB)
     |████████████████████████████████| 7.3 MB 1.5 MB/s            
  Using cached nltk-3.5-py3-none-any.whl
  Using cached numpy-1.18.5-cp38-cp38-macosx_10_9_x86_64.whl (15.1 MB)
  Using cached opencv_python-4.5.1.48-cp38-cp38-macosx_10_13_x86_64.whl (40.3 MB)
  Using cached Pillow-7.1.2-cp38-cp38-macosx_10_10_x86_64.whl (2.2 MB)
     |████████████████████████████████| 7.9 MB 2.7 MB/s            
     |████████████████████████████████| 33.0 MB 3.2 MB/s            
  Using cached torch-1.8.1-cp38-none-macosx_10_9_x86_64.whl (119.6 MB)
  Using cached torchaudio-0.8.1-cp38-cp38-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached torchinfo-0.0.8-py3-none-any.whl (16 kB)
  Using cached torchtext-0.9.1-cp38-cp38-macosx_10_9_x86_64.whl (1.6 MB)
  Using cached torchvision-0.9.1-cp38-cp38-macos

# 文本分类任务

在本模块中，我们将从基于 **AG_NEWS** 数据集的简单文本分类任务开始，该任务将新闻标题分类为 4 个类别之一：World、Sports、Business 和 Sci/Tech。 该数据集内置于 [`torchtext`](https://github.com/pytorch/text) 模块中，因此我们可以轻松访问它。

In [2]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

train.csv: 29.5MB [00:04, 6.29MB/s]                            
test.csv: 1.86MB [00:00, 4.27MB/s]                          


这里，`train_dataset` 和 `test_dataset` 包含迭代器，它们分别返回标签对（类别数）和文本，例如：

In [3]:
next(train_dataset)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

因此，让我们打印出数据集中的前 10 个新标题：

In [4]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace ba

因为数据集是迭代器，如果我们想多次使用数据，我们需要将其转换为列表：

In [5]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

现在我们需要将文本转换为可以表示为张量的**数字**。 如果我们想要词级表示，我们需要做两件事：
* 使用 **tokenizer** 将文本拆分为 **tokens**
* 构建这些标记的 **词汇**。

In [6]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [7]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

使用词汇表，我们可以轻松地将标记化的字符串编码为一组数字：

In [8]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95812


[283, 2321, 5, 337, 19, 1301, 2357]

## 词袋文本表示

因为单词代表意义，有时我们可以通过查看单个单词来理解文本的含义，而不管它们在句子中的顺序如何。 例如，在对新闻进行分类时，像*weather*、*snow* 这样的词可能表示*weather forecast*，而像*stocks*、*dollar* 这样的词将计入*financial news*。

**Bag of Words** (BoW) 向量表示是最常用的传统向量表示。 每个词都链接到一个向量索引，向量元素包含一个词在给定文档中出现的次数。

![图像显示了词袋向量表示如何在内存中表示。](./images/bag-of-words-example.png)

> **注意**：您还可以将 BoW 视为文本中单个单词的所有单热编码向量的总和。

下面是如何使用 Scikit Learn python 库生成词袋表示的示例：

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()



array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

要从 AG_NEWS 数据集的向量表示中计算词袋向量，我们可以使用以下函数：

In [10]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([0., 0., 2.,  ..., 0., 0., 0.])


> **注意：** 这里我们使用全局`vocab_size` 变量来指定词汇表的默认大小。 由于通常词汇量非常大，我们可以将词汇量限制为最常用的词。 尝试降低 `vocab_size` 值并运行下面的代码，看看它如何影响准确性。 您应该期待一些准确度下降，但不会显着下降，以代替更高的性能。

## 训练 BoW 分类器

现在我们已经学会了如何构建文本的词袋表示，让我们在它之上训练一个分类器。 首先，我们需要以这种方式转换我们的训练数据集，将所有位置向量表示转换为词袋表示。 这可以通过将 `bowify` 函数作为 `collate_fn` 参数传递给标准 Torch `DataLoader` 来实现：

现在让我们定义一个包含一个线性层的简单分类器神经网络。 输入向量的大小等于`vocab_size`，输出大小对应于类的数量（4）。 因为我们是在解决分类任务，所以最终的激活函数是`LogSoftmax()`。

In [11]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

现在让我们定义一个包含一个线性层的简单分类器神经网络。 输入向量的大小等于`vocab_size`，输出大小对应于类的数量（4）。 因为我们是在解决分类任务，所以最终的激活函数是`LogSoftmax()`。

In [12]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

现在我们将定义标准的 PyTorch 训练循环。 因为我们的数据集非常大，出于教学目的，我们只会训练一个 epoch，有时甚至少于一个 epoch（指定 epoch_size 参数允许我们限制训练）。 我们还会在训练过程中报告累积的训练准确率； 报告频率是使用`report_freq` 参数指定的。

In [13]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [14]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8096875
6400: acc=0.84015625
9600: acc=0.8528125
12800: acc=0.856875


(0.026818861076826735, 0.8601412579957356)

## BiGrams、TriGrams 和 N-Grams

词袋方法的一个限制是某些词是多词表达的一部分，例如，“热狗”一词与其他上下文中的词“热”和“狗”具有完全不同的含义。 如果我们总是用相同的向量表示单词“hot”和“dog”，它会混淆我们的模型。

为了解决这个问题，**N-gram 表示**经常用于文档分类方法，其中每个词、双词或三词的频率是训练分类器的有用特征。 例如，在双元组表示中，除了原始单词之外，我们还会将所有单词对添加到词汇表中。

下面是一个如何使用 Scikit Learn 生成词表示的二元词袋的示例：


In [15]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

N-gram 方法的主要缺点是词汇量开始增长得非常快。 在实践中，我们需要将 N-gram 表示与一些降维技术相结合，例如 *embeddings*，我们将在下一个单元中讨论。

要在我们的 **AG News** 数据集中使用 N-gram 表示，我们需要构建特殊的 ngram 词汇表：

In [16]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.Vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308844


然后我们可以使用与上面相同的代码来训练分类器，但是，它的内存效率非常低。 在下一个单元中，我们将使用嵌入训练二元分类器。

> **注意：** 您只能保留在文本中出现次数超过指定次数的那些 ngram。 这将确保忽略不常见的二元组，并显着降低维度。 为此，将 `min_freq` 参数设置为更高的值，并观察词汇变化的长度。

##词频逆文档频率TF-IDF

在 BoW 表示中，单词出现的权重均匀，而与单词本身无关。但是，很明显，与专业术语相比，*a*、*in* 等频繁出现的词对分类的重要性要小得多。事实上，在大多数 NLP 任务中，有些词比其他词更相关。

**TF-IDF** 代表**词频-逆文档频率**。它是词袋的一种变体，其中使用浮点值而不是指示单词在文档中出现的二进制 0/1 值，该值与语料库中单词出现的频率有关。

更正式地，文档$j$中单词$i$的权重$w_{ij}$定义为：
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
在哪里
* $tf_{ij}$是$j$中$i$出现的次数，即我们之前看到的BoW值
* $N$ 是集合中的文档数
* $df_i$ 是整个集合中包含单词 $i$ 的文档数

TF-IDF 值 $w_{ij}$ 与单词在文档中出现的次数成正比增加，并被包含该单词的语料库中的文档数所抵消，这有助于调整某些单词出现的事实比其他人更频繁。例如，如果该词出现在集合中的*每个* 文档中，$df_i=N$ 和 $w_{ij}=0$，这些词将被完全忽略。

您可以使用 Scikit Learn 轻松创建文本的 TF-IDF 矢量化：

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

然而，即使 TF-IDF 表示为不同的单词提供频率权重，它们也无法表示含义或顺序。 正如著名语言学家 J. R. Firth 在 1935 年所说的那样，“一个词的完整意义总是与语境相关的，没有语境就不能认真研究意义。”。 我们将在后面的单元中学习如何使用语言建模从文本中捕获上下文信息。